### Data Visualization of Suzhou Coronavirus cases

In [4]:
#-*- coding: utf-8 -*-
import pandas as pd
import json
from urllib.request import urlopen, quote

In [160]:
df = pd.read_excel('cases.xlsx')
df.head()
region = [loc[:5] for loc in df.loc[:, '小区']]
from collections import Counter
region_count = dict(Counter(region))

In [6]:
def getlnglat(address):
    url = 'http://api.map.baidu.com/geocoding/v3/'
    output = 'json'
    ak = 'HWEdwfifWhtfGd3QxCCKgpGFpYrFT4bb'
    address = quote(address)
    url = url + '?' + 'address=' + address + '&output=' + output + '&ak=' + ak
    req = urlopen(url)
    res  = req.read().decode()
    temp = json.loads(res)
    lng = temp['result']['location']['lng']
    lat = temp['result']['location']['lat']
    type = temp['result']['level']
    return lng, lat, type

In [131]:
data = []
for i in df.index:
    loc = df.loc[i, '小区']
    lng, lat, type = getlnglat(loc)
    df.loc[i, '经度'] = lng       
    df.loc[i, '纬度'] = lat
    data.append({'name': loc, 'value': [lng, lat, 1]})

In [8]:
from pyecharts import options as opts
from pyecharts.charts import BMap, Geo, Page
from pyecharts.globals import ChartType, SymbolType

In [165]:
def add_address_json() -> Geo:

    count = [1]*len(data)
    
    locs = []
    json_data = {}
    
    for i, v in enumerate(data):
        name = v['name']
        locs.append(name)
        json_data[name] = [v['value'][0], v['value'][1]]
    
    json_str = json.dumps(json_data, ensure_ascii=False, indent=4)
    with open('test_data.json', 'w', encoding='utf-8') as json_file:
        json_file.write(json_str)
        
    c = (
        Geo()
        .add_schema(maptype='苏州') 
        .add_coordinate_json(json_file='test_data.json')
        .add("", data_pair =[list(z) for z in zip(locs, count)], symbol_size = 30, large_threshold = 2000, symbol="pin")
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(title_opts=opts.TitleOpts(title="苏州疫情地点"))
        )
    return c

In [166]:
add_address_json().render_notebook()

In [161]:
add_address_json().render()

'C:\\Users\\ashle\\Codes\\Mine\\render.html'

In [196]:
def add_address_json_bmap() -> BMap:
    
    BAIDU_AK = 'HWEdwfifWhtfGd3QxCCKgpGFpYrFT4bb'
    sz = Geo().get_coordinate('苏州')

    count = [1]*len(data)
    
    locs = []
    json_data = {}
    
    for i, v in enumerate(data):
        if v['name'][4] == '市':
            name = (v['name'][2:])
        else:
            idx = v['name'].find('区')
            name = (v['name'][idx+1:])
        locs.append(name)
        json_data[name] = [v['value'][0], v['value'][1]]
    
    json_str = json.dumps(json_data, ensure_ascii=False, indent=4)
    with open('test_data.json', 'w', encoding='utf-8') as json_file:
        json_file.write(json_str)
        
    c = (
        BMap()
        .add_schema(
            baidu_ak=BAIDU_AK,
            center=sz,
            zoom=10,
            map_style={
                "styleJson": [
                    {
                        "featureType": "water",
                        "elementType": "all",
                        "stylers": {"color": "#d1d1d1"},
                    },
                    {
                        "featureType": "land",
                        "elementType": "all",
                        "stylers": {"color": "#f3f3f3"},
                    },
                    {
                        "featureType": "railway",
                        "elementType": "all",
                        "stylers": {"visibility": "off"},
                    },
                    {
                        "featureType": "highway",
                        "elementType": "all",
                        "stylers": {"color": "#999999"},
                    },
                    {
                        "featureType": "highway",
                        "elementType": "labels",
                        "stylers": {"visibility": "off"},
                    },
                    {
                        "featureType": "arterial",
                        "elementType": "geometry",
                        "stylers": {"color": "#fefefe"},
                    },
                    {
                        "featureType": "arterial",
                        "elementType": "geometry.fill",
                        "stylers": {"color": "#fefefe"},
                    },
                    {
                        "featureType": "poi",
                        "elementType": "all",
                        "stylers": {"visibility": "off"},
                    },
                    {
                        "featureType": "green",
                        "elementType": "all",
                        "stylers": {"visibility": "off"},
                    },
                    {
                        "featureType": "subway",
                        "elementType": "all",
                        "stylers": {"visibility": "off"},
                    },
                    {
                        "featureType": "manmade",
                        "elementType": "all",
                        "stylers": {"color": "#d1d1d1"},
                    },
                    {
                        "featureType": "local",
                        "elementType": "all",
                        "stylers": {"color": "#d1d1d1"},
                    },
                    {
                        "featureType": "arterial",
                        "elementType": "labels",
                        "stylers": {"visibility": "off"},
                    },
                    {
                        "featureType": "boundary",
                        "elementType": "all",
                        "stylers": {"color": "#fefefe"},
                    },
                    {
                        "featureType": "building",
                        "elementType": "all",
                        "stylers": {"color": "#d1d1d1"},
                    },
                    {
                        "featureType": "label",
                        "elementType": "labels.text.fill",
                        "stylers": {"color": "rgba(0,0,0,0)"},
                    },
                ]
            },
        )
        .add_coordinate_json(json_file='test_data.json')
        .add(
            "苏州确诊病例涉及地点",
            data_pair =[list(z) for z in zip(locs, count)],
            label_opts=opts.LabelOpts(is_show=False, formatter="{b}"),
        )
        .add_control_panel(
            scale_control_opts=opts.BMapScaleControlOpts(),
            overview_map_opts=opts.BMapOverviewMapControlOpts(is_open=True),
            navigation_control_opts=opts.BMapNavigationControlOpts(),
            geo_location_control_opts=opts.BMapGeoLocationControlOpts()
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="苏州疫情地点"))
    )
    return c

In [197]:
add_address_json_bmap().render()

'C:\\Users\\ashle\\Codes\\Mine\\render.html'